In [ ]:
import requests
import gzip
import urllib.request
import json
from bs4 import BeautifulSoup
gameList = ['golf','boxing','mlb','nba','nfl']
#gameList = ['nfl']

for game in gameList:
    lst = []
    count = 0 
    link = "https://index.commoncrawl.org/CC-MAIN-2019-13-index?url=www.espn.com%2F"+game+"*&output=json"
    resp = requests.get(link)
    string = resp.content.decode('utf-8')
    s = string.split('\n')
    l = len(s)
    fname = game+"_full.txt"
    f=open(fname, "a+")
    for c in range(l-2,0,-1):
        json_obj = json.loads(s[c])
        offset = int(json_obj["offset"])
        length = int(json_obj["length"])
        url = json_obj["url"]
        if url not in lst:
            lst.append(url)
            end = offset + length -1
            filename = json_obj["filename"]
            request = urllib.request.Request(
                "https://commoncrawl.s3.amazonaws.com/"+filename,headers={'Range': 'bytes={}-{}'.format(offset, end)})
            response = urllib.request.urlopen(request)
            gzipFile = gzip.GzipFile(fileobj=response)
            var = gzipFile.read()
            #making soup
            soup = BeautifulSoup(var, "html.parser")
            m = soup.find_all('p')
            for i in m:
                f.write(i.text)
            f.write('\n')